In [ ]:
import cx_Oracle
import tkinter as tk
from tkinter import ttk, messagebox

try:
    conn = cx_Oracle.connect("system", "password", "localhost/XE")
    cursor = conn.cursor()
except cx_Oracle.DatabaseError as e:
    print("There was a problem connecting to Oracle:", e)
    messagebox.showerror("Database Error", f"There was a problem connecting to Oracle: {e}")
    conn = None
    cursor = None

TABLES = ["Admin", "Students", "Teachers", "Grades", "Subjects", "Classes"]


def view_data(table):
    if cursor is None:
        messagebox.showerror("Error", "Database connection not established.")
        return

    try:
        cursor.execute(f"SELECT * FROM {table}")
        rows = cursor.fetchall()
        if rows:
            view_window = tk.Toplevel(root)
            view_window.title(f"View {table} Data")
            headers = [desc[0] for desc in cursor.description]

            for col, header in enumerate(headers):
                tk.Label(view_window, text=header, relief="solid", width=20).grid(row=0, column=col)

            for row_num, row in enumerate(rows, start=1):
                for col_num, value in enumerate(row):
                    tk.Label(view_window, text=value, relief="solid", width=20).grid(row=row_num, column=col_num)
        else:
            messagebox.showinfo("No Data", f"No data found in {table}.")
    except cx_Oracle.DatabaseError as e:
        messagebox.showerror("Error", f"Error viewing data from {table}: {e}")


def insert_data(table):
    if cursor is None:
        messagebox.showerror("Error", "Database connection not established.")
        return

    def submit_data():
        data = [entry.get() for entry in entries]
        columns = ", ".join(column_names)
        placeholders = ", ".join([":" + str(i + 1) for i in range(len(data))])
        insert_query = f"INSERT INTO {table} ({columns}) VALUES ({placeholders})"
        try:
            cursor.execute(insert_query, data)
            conn.commit()
            messagebox.showinfo("Success", f"Data inserted successfully into {table}.")
            insert_window.destroy()
        except cx_Oracle.DatabaseError as e:
            messagebox.showerror("Error", f"Error inserting data into {table}: {e}")

    insert_window = tk.Toplevel(root)
    insert_window.title(f"Insert Data into {table}")

    cursor.execute(f"SELECT * FROM {table} WHERE ROWNUM = 1")
    column_names = [desc[0] for desc in cursor.description]
    entries = []

    for idx, col_name in enumerate(column_names):
        tk.Label(insert_window, text=col_name).grid(row=idx, column=0)
        entry = tk.Entry(insert_window)
        entry.grid(row=idx, column=1)
        entries.append(entry)

    tk.Button(insert_window, text="Submit", command=submit_data).grid(row=len(column_names), column=1)


def delete_data(table):
    if cursor is None:
        messagebox.showerror("Error", "Database connection not established.")
        return

    def submit_delete():
        condition_value = condition_entry.get()
        delete_query = f"DELETE FROM {table} WHERE {condition_column.get()} = :1"
        try:
            cursor.execute(delete_query, [condition_value])
            conn.commit()
            messagebox.showinfo("Success", f"Data deleted successfully from {table}.")
            delete_window.destroy()
        except cx_Oracle.DatabaseError as e:
            messagebox.showerror("Error", f"Error deleting data from {table}: {e}")

    delete_window = tk.Toplevel(root)
    delete_window.title(f"Delete Data from {table}")

    cursor.execute(f"SELECT * FROM {table} WHERE ROWNUM = 1")
    column_names = [desc[0] for desc in cursor.description]

    tk.Label(delete_window, text="Condition Column:").grid(row=0, column=0)
    condition_column = tk.StringVar(value=column_names[0])
    ttk.Combobox(delete_window, textvariable=condition_column, values=column_names).grid(row=0, column=1)

    tk.Label(delete_window, text="Condition Value:").grid(row=1, column=0)
    condition_entry = tk.Entry(delete_window)
    condition_entry.grid(row=1, column=1)

    tk.Button(delete_window, text="Delete", command=submit_delete).grid(row=2, column=1)


def update_data(table):
    if cursor is None:
        messagebox.showerror("Error", "Database connection not established.")
        return

    def submit_update():
        try:
            condition_value = condition_entry.get()
            update_values = [entry.get() for entry in entries]
            update_values.append(condition_value)
            update_query = f"UPDATE {table} SET {', '.join([f'{col} = :{idx + 1}' for idx, col in enumerate(column_names)])} WHERE {condition_column.get()} = :{len(update_values)}"
            cursor.execute(update_query, update_values)
            conn.commit()
            messagebox.showinfo("Success", f"Data updated successfully in {table}.")
            update_window.destroy()
        except cx_Oracle.DatabaseError as e:
            messagebox.showerror("Error", f"Error updating data in {table}: {e}")

    update_window = tk.Toplevel(root)
    update_window.title(f"Update Data in {table}")

    cursor.execute(f"SELECT * FROM {table} WHERE ROWNUM = 1")
    column_names = [desc[0] for desc in cursor.description]
    entries = []

    tk.Label(update_window, text="Condition Column:").grid(row=0, column=0)
    condition_column = tk.StringVar(value=column_names[0])
    ttk.Combobox(update_window, textvariable=condition_column, values=column_names).grid(row=0, column=1)

    tk.Label(update_window, text="Condition Value:").grid(row=1, column=0)
    condition_entry = tk.Entry(update_window)
    condition_entry.grid(row=1, column=1)

    tk.Label(update_window, text="Set Values:").grid(row=2, column=0, columnspan=2)
    for idx, col_name in enumerate(column_names):
        tk.Label(update_window, text=col_name).grid(row=idx + 3, column=0)
        entry = tk.Entry(update_window)
        entry.grid(row=idx + 3, column=1)
        entries.append(entry)

    tk.Button(update_window, text="Submit", command=submit_update).grid(row=len(column_names) + 3, column=1)


def commit_and_exit():
    if cursor is None:
        messagebox.showerror("Error", "Database connection not established.")
        return

    try:
        conn.commit()
        messagebox.showinfo("Success", "Changes committed successfully. Exiting application.")
    except cx_Oracle.DatabaseError as e:
        messagebox.showerror("Error", f"Error committing changes: {e}")
    finally:
        if conn is not None:
            conn.close()
        root.destroy()


root = tk.Tk()
root.title("Database Management System")
root.geometry("800x450")

table_frame = tk.Frame(root, width=200, bg="lightgray")
table_frame.pack(side="left", fill="y")

tk.Label(table_frame, text="Tables", bg="lightgray", font=("Arial", 14)).pack(pady=10)
for table in TABLES:
    ttk.Button(table_frame, text=table, command=lambda t=table: table_var.set(t)).pack(pady=5, padx=10, fill="x")

action_frame = tk.Frame(root, bg="white")
action_frame.pack(side="right", expand=True, fill="both")

table_var = tk.StringVar(value=TABLES[0])

tk.Label(action_frame, text="Select Action", font=("Arial", 16), bg="white").pack(pady=20)

ttk.Button(action_frame, text="View Data", command=lambda: view_data(table_var.get())).pack(pady=10, padx=20)
ttk.Button(action_frame, text="Insert Data", command=lambda: insert_data(table_var.get())).pack(pady=10, padx=20)
ttk.Button(action_frame, text="Update Data", command=lambda: update_data(table_var.get())).pack(pady=10, padx=20)
ttk.Button(action_frame, text="Delete Data", command=lambda: delete_data(table_var.get())).pack(pady=10, padx=20)
ttk.Button(action_frame, text="Commit and Exit", command=commit_and_exit).pack(pady=10, padx=20)

root.mainloop()

if conn is not None:
    conn.close()
